In [43]:
#Importing Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
import gensim
from gensim.models import Word2Vec

In [3]:
# Reading JSON files
df = pd.read_json("D:/Major Project/datasets/SC Dataset-20240615T035003Z-001/SC Dataset/newstext_1.json")
df.sample(5)

,newsId,newsSource,newsText
6415,N1381453242,http://www.gorkhapatraonline.com/,टिकट बुकिङ अनिश्चित माछापालनको रकम सामाजिक क्ष...
3933,N1380617384,http://www.gorkhapatraonline.com/,भूमिगत पानी जथाभावी निकाल्दा जोखिम एमाले–माओवा...
4313,N1380647056,http://www.gorkhapatraonline.com/,सरकारमा सहभागी हुन संविधान संशोधन फोरमको शर्त ...
6695,N1382250963,http://www.gorkhapatraonline.com/,निर्वाचन र जात्रा एकै दिन राष्ट्रपति भण्डारी श...
2015,N1381483096,http://www.gorkhapatraonline.com/,स्वास्थ्य टोली बाढी प्रभावित क्षेत्रमा नयाँ दि...


In [4]:
corpus = ''.join(df['newsText'])
len(corpus)

22140722

In [5]:
corpus[:500]

'विकल्परहित दोस्रो चरण शिवकुमार भट्टराई भनिन्छ– राजनीति सम्भावनाको खेल हो ।\xa0 तर, राजनीति सम्भावनासँगै कल्पनाबाहिरको प्रपञ्चको पुलिन्दासमेत बन्न पुगेको छ ।\xa0 नेपालको पछिल्लो राजनीतिक परिघटनाले यो अवस्थाको स्पष्ट चित्रण गरेको छ ।\xa0 सन्दर्भ सत्ता हस्तान्तरणको हो ।\xa0 गएको साउन १९ गते नेपाली काँग्रेससँग भद्रसहमति गरी स्थानीय तहको निर्वाचनको कार्यभारसहित पहिलो चरणमा पुष्पकमल दाहाल ‘प्रचण्ड’को नेतृत्वमा संयुक्त सरकार बनाउने भद्रसहमतिअनुसार सरकार बन्यो ।\xa0 त्यसपछि प्रदेश र केन्द्रको निर्वाचन गर्न काँग्रेस सभ'

In [8]:
# # Save the corpus to a text file
# with open('raw_corpus.txt', 'w', encoding='utf-8') as file:
#     file.write(corpus)

In [6]:
# Load the corpus from the text file
with open('raw_corpus.txt', 'r', encoding='utf-8') as file:
    loaded_corpus = file.read()

In [7]:
# Verify by printing the first 500 characters
print(loaded_corpus[:500])

विकल्परहित दोस्रो चरण शिवकुमार भट्टराई भनिन्छ– राजनीति सम्भावनाको खेल हो ।  तर, राजनीति सम्भावनासँगै कल्पनाबाहिरको प्रपञ्चको पुलिन्दासमेत बन्न पुगेको छ ।  नेपालको पछिल्लो राजनीतिक परिघटनाले यो अवस्थाको स्पष्ट चित्रण गरेको छ ।  सन्दर्भ सत्ता हस्तान्तरणको हो ।  गएको साउन १९ गते नेपाली काँग्रेससँग भद्रसहमति गरी स्थानीय तहको निर्वाचनको कार्यभारसहित पहिलो चरणमा पुष्पकमल दाहाल ‘प्रचण्ड’को नेतृत्वमा संयुक्त सरकार बनाउने भद्रसहमतिअनुसार सरकार बन्यो ।  त्यसपछि प्रदेश र केन्द्रको निर्वाचन गर्न काँग्रेस सभ


In [8]:
len(loaded_corpus)

22140722

In [9]:
# function to count number of words in each records
def count_words(text):
    return len(text.split())

# fucntion to count no.of sentences ..delimeters: [| ! ?]
def sentence_count(text):
    sentences = re.split(r'[।!?]',text)
    sentences = [s for s in sentences if s.strip()]   # Remove emtpy string from list
    return len(sentences)

In [10]:
count_word = count_words(loaded_corpus)
count_word

3350330

In [11]:
sen_count = sentence_count(corpus)
sen_count

181079

In [12]:
def normalize_text(text):
    # Remove punctuation and non-Devanagari characters
    text = re.sub(r'[^\u0900-\u097F\s।]', '', text)
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def preprocess_text(text):
    # Normalize text
    normalized_text = normalize_text(text)
    sentences = normalized_text.split('।') # !?

#     # Tokenize words in each sentence
    tokenized_sentences = [word_tokenize(sentence) for sentence in sentences if len(sentence.split())>=3]
    return tokenized_sentences

In [23]:
processed_text = preprocess_text(loaded_corpus)

In [13]:
len(processed_text)

NameError: name 'processed_text' is not defined

In [24]:
print(processed_text[:500])

[['विकल्परहित', 'दोस्रो', 'चरण', 'शिवकुमार', 'भट्टराई', 'भनिन्छ', 'राजनीति', 'सम्भावनाको', 'खेल', 'हो'], ['तर', 'राजनीति', 'सम्भावनासँगै', 'कल्पनाबाहिरको', 'प्रपञ्चको', 'पुलिन्दासमेत', 'बन्न', 'पुगेको', 'छ'], ['नेपालको', 'पछिल्लो', 'राजनीतिक', 'परिघटनाले', 'यो', 'अवस्थाको', 'स्पष्ट', 'चित्रण', 'गरेको', 'छ'], ['सन्दर्भ', 'सत्ता', 'हस्तान्तरणको', 'हो'], ['गएको', 'साउन', '१९', 'गते', 'नेपाली', 'काँग्रेससँग', 'भद्रसहमति', 'गरी', 'स्थानीय', 'तहको', 'निर्वाचनको', 'कार्यभारसहित', 'पहिलो', 'चरणमा', 'पुष्पकमल', 'दाहाल', 'प्रचण्डको', 'नेतृत्वमा', 'संयुक्त', 'सरकार', 'बनाउने', 'भद्रसहमतिअनुसार', 'सरकार', 'बन्यो'], ['त्यसपछि', 'प्रदेश', 'र', 'केन्द्रको', 'निर्वाचन', 'गर्न', 'काँग्रेस', 'सभापति', 'शेरबहादुर', 'देउवा', 'नेतृत्वको', 'सरकार', 'निर्माण', 'गर्ने', 'भनिएको', 'हो'], ['पहिलो', 'चरणको', 'कार्यभार', 'लगभग', 'पूरा', 'गरेको', 'प्रचण्ड', 'सरकारले', 'पूर्वसहमतिबमोजिम', 'राजीनामा', 'गर्ने', 'तयारी', 'गरेपछि', 'प्रमुख', 'प्रतिपक्ष', 'एमालेमा', 'एकाएक', 'प्रचण्डप्रति', 'प्रचण्ड', 'सहानुभूतिको', 'भू

In [28]:
type(processed_text)

list

In [44]:
import pickle

In [31]:
# Save the list to a file using pickle
with open('processed_list.pkl', 'wb') as file:
    pickle.dump(processed_text, file)

print("Tokenized list saved to 'processed_list.pkl'")

Tokenized list saved to 'processed_list.pkl'


In [16]:
# Load the list from the file using pickle
with open('processed_list.pkl', 'rb') as file:
    loaded_processed_list = pickle.load(file)

# Print the loaded list
print(loaded_processed_list[:500])

[['विकल्परहित', 'दोस्रो', 'चरण', 'शिवकुमार', 'भट्टराई', 'भनिन्छ', 'राजनीति', 'सम्भावनाको', 'खेल', 'हो'], ['तर', 'राजनीति', 'सम्भावनासँगै', 'कल्पनाबाहिरको', 'प्रपञ्चको', 'पुलिन्दासमेत', 'बन्न', 'पुगेको', 'छ'], ['नेपालको', 'पछिल्लो', 'राजनीतिक', 'परिघटनाले', 'यो', 'अवस्थाको', 'स्पष्ट', 'चित्रण', 'गरेको', 'छ'], ['सन्दर्भ', 'सत्ता', 'हस्तान्तरणको', 'हो'], ['गएको', 'साउन', '१९', 'गते', 'नेपाली', 'काँग्रेससँग', 'भद्रसहमति', 'गरी', 'स्थानीय', 'तहको', 'निर्वाचनको', 'कार्यभारसहित', 'पहिलो', 'चरणमा', 'पुष्पकमल', 'दाहाल', 'प्रचण्डको', 'नेतृत्वमा', 'संयुक्त', 'सरकार', 'बनाउने', 'भद्रसहमतिअनुसार', 'सरकार', 'बन्यो'], ['त्यसपछि', 'प्रदेश', 'र', 'केन्द्रको', 'निर्वाचन', 'गर्न', 'काँग्रेस', 'सभापति', 'शेरबहादुर', 'देउवा', 'नेतृत्वको', 'सरकार', 'निर्माण', 'गर्ने', 'भनिएको', 'हो'], ['पहिलो', 'चरणको', 'कार्यभार', 'लगभग', 'पूरा', 'गरेको', 'प्रचण्ड', 'सरकारले', 'पूर्वसहमतिबमोजिम', 'राजीनामा', 'गर्ने', 'तयारी', 'गरेपछि', 'प्रमुख', 'प्रतिपक्ष', 'एमालेमा', 'एकाएक', 'प्रचण्डप्रति', 'प्रचण्ड', 'सहानुभूतिको', 'भू

In [35]:
# Train Word2Vec model
model = Word2Vec(
    sentences=loaded_processed_list,
    vector_size=200,
    window=5,
    min_count=5,
    workers=4,
    epochs=10,
    sg=1,
    negative=10
)



In [36]:
# Save the model
model.save("nepali_word2vec_1.model")

In [18]:
len(loaded_processed_list)

178362

In [17]:
model = Word2Vec.load("nepali_word2vec_1.model")

In [19]:
# Resume training for JSON_2

In [20]:
def read_json_file(path):
    df = pd.read_json(path)
    return df

In [29]:
def merge_rec(df1):
    corpus = ''.join(df1['newsText'])
    # Save the corpus to a text file
    with open('2_corpus.txt', 'w', encoding='utf-8') as file:
        file.write(corpus)
    return corpus

In [31]:
# Load the corpus from the text file
with open('2_corpus.txt', 'r', encoding='utf-8') as file:
    loaded_corpus = file.read()

In [30]:
def normalize_text(text):
    # Remove punctuation and non-Devanagari characters
    text = re.sub(r'[^\u0900-\u097F\s।]', '', text)
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def preprocess_text(text):
    # Normalize text
    normalized_text = normalize_text(text)
    sentences = normalized_text.split('।') # !?

#     # Tokenize words in each sentence
    tokenized_sentences = [word_tokenize(sentence) for sentence in sentences if len(sentence.split())>=3]
    return tokenized_sentences

In [33]:
processed_text_2 = preprocess_text(loaded_corpus)

In [37]:
with open('processed_text_2.pkl','wb') as file:
    pickle.dump(processed_text_2,file)
print("Tokenized list saved to 'processed_text_2.pkl'")


Tokenized list saved to 'processed_text_2.pkl'


In [38]:
# Load the list from the file using pickle
with open('processed_text_2.pkl', 'rb') as file:
    loaded_processed_list = pickle.load(file)

# Print the loaded list
print(loaded_processed_list[:500])

[['जात्रा', 'आज', 'सकिँदै', 'वरगाछीरोडशेष', 'सडक', 'कालोपत्र', 'हुने', 'विराटनगर', 'मंसिर', '२', 'गते'], ['विराटनगर', 'महानगरपालिकाको', 'बरगाछीदेखि', 'रोडशेष', 'चोकसम्मको', 'तीन', 'किमी', 'धुलाम्मे', 'सडक', 'कालोपत्र', 'भई', 'समस्या', 'समाधान', 'हुने', 'भएको', 'छ'], ['महानगरपालिका', 'र', 'व्यापारिक', 'मार्ग', 'विस्तार', 'आयोजना', '९छ', 'लेन०', 'को', 'प्रतिनिधिको', 'टोलीबीच', 'शुक्रबार', 'सडक', 'समस्या', 'चाँडै', 'समाधान', 'गर्ने', 'सहमति', 'भएको', 'छ'], ['अहिले', 'तत्कालका', 'लागि', 'बरगाछीदेखि', 'रोडशेष', 'चोकसम्म', 'तीन', 'किमी', 'र', 'नौ', 'मी', 'चौडा', 'सडकमा', 'द्रुत', 'गतिमा', 'कालोपत्र', 'गर्ने', 'सहमति', 'भएको', 'महानगरपालिकाका', 'प्रमुख', 'भीम', 'पराजुलीले', 'जानकारी', 'दिनुभयो'], ['उक्त', 'सडक', 'कालोपत्र', 'भएसँगै', 'महानगरवासीलाई', 'आवतजावत', 'तथा', 'सवारी', 'सञ्चालनमा', 'केही', 'हदसम्म', 'सहज', 'हुने', 'अनुमान', 'गरिएको', 'छ'], ['सडक', 'कालोपत्र', 'गर्ने', 'सम्बन्धमा', 'भएको', 'छलफलमा', 'महानगर', 'प्रमुख', 'पराजुली', 'प्रशासकीय', 'प्रमुख', 'प्रदीप', 'निरौला', 'इ', 'घनेश्या

In [39]:
model.build_vocab(loaded_processed_list, update=True)

In [42]:
model.build_vocab(loaded_processed_list, update=True)
model.train(loaded_processed_list, total_examples=len(loaded_processed_list), epochs=model.epochs)


(28817275, 32848240)

In [44]:
# Save the updated model
model.save("word2vec_model_2.model")

In [45]:
# Retrieve and print the training loss
training_loss = model.get_latest_training_loss()
print("Training Loss:", training_loss)

Training Loss: 0.0


In [43]:
len(loaded_processed_list)

188829

In [46]:
# Print cumulative and effective word count
print("Cumulative Word Count:", model.corpus_count)
print("Effective Word Count:", model.corpus_total_words)

Cumulative Word Count: 188829
Effective Word Count: 3284824


In [47]:
model = Word2Vec.load("word2vec_model_2.model")

In [30]:
corpus = merge_rec(df1)

In [28]:
len(corpus)

23029822

In [21]:
path = "D:/Major Project/datasets/SC Dataset-20240615T035003Z-001/SC Dataset/newstext_2.json"

In [22]:
df1 = read_json_file(path)
df1.head()

,newsId,newsSource,newsText
0,N1381386253,http://www.gorkhapatraonline.com/,जात्रा आज सकिँदै वरगाछी–रोडशेष सडक कालोपत्र हु...
1,N1381386251,http://www.gorkhapatraonline.com/,जात्रा आज सकिँदै नवीन कलाश्री पुरस्कार घोषित प...
2,N1381386216,http://www.gorkhapatraonline.com/,जात्रा आज सकिँदै लमजुङ सदरमुकाम बेँसीशहरमा बम ...
3,N1381386252,http://www.gorkhapatraonline.com/,जात्रा आज सकिँदै आठराई गाउँपालिकाको बजेट रु ३२...
4,N1381386129,http://www.gorkhapatraonline.com/,जात्रा आज सकिँदै पर्यटन प्रवद्र्धनको दिशा (सम...


In [2]:
###################################TEST#######################


In [24]:
# Reading JSON files
df_3 = pd.read_json("D:/Major Project/datasets/SC Dataset-20240615T035003Z-001/SC Dataset/newstext_3.json")
df_4 = pd.read_json("D:/Major Project/datasets/SC Dataset-20240615T035003Z-001/SC Dataset/newstext_4.json")
df_5 = pd.read_json("D:/Major Project/datasets/SC Dataset-20240615T035003Z-001/SC Dataset/newstext_5.json")
df_6 = pd.read_json("D:/Major Project/datasets/SC Dataset-20240615T035003Z-001/SC Dataset/newstext_6.json")
df_7 = pd.read_json("D:/Major Project/datasets/SC Dataset-20240615T035003Z-001/SC Dataset/newstext_7.json")
df_8 = pd.read_json("D:/Major Project/datasets/SC Dataset-20240615T035003Z-001/SC Dataset/newstext_8.json")

In [34]:
df_4.sample(4)

,newsId,newsSource,newsText
214,N407889656,https://ekantipur.com/,"‘गाउँमा सिंह आयो, दरबार उतै छुट्यो’ सैद्धान्ति..."
2314,N1072583054,https://ekantipur.com/,अबको निकास संविधान संशोधन प्रत्येक नौजवानले सक...
6855,P1219343997,https://ekantipur.com/,ब्रबिम कुमार काठमाडौँ — 'कता गायब ?' ठ्याक्कै ...
9748,N1838658976,https://ekantipur.com/,सत्ता परिवर्तन : कहाँ चुके दलहरू ? केन्द्रमा ज...


In [35]:
def merge_rec(df):
    corpus = ''.join(df['newsText'])
    return corpus

In [36]:
corpus_5 = merge_rec(df_5)

In [37]:
processed_text_5 = preprocess_text(corpus_5)

In [38]:
model = Word2Vec.load("Updated_word2vec_model_4.model")

In [39]:

model.build_vocab(processed_text_4, update=True)
model.train(processed_text_4, total_examples=len(processed_text_4), epochs=model.epochs)
# Save the updated model
model.save("Updated_word2vec_model_4.model")

In [ ]:
vocab_size = len(model.wv.key_to_index)
print(f"New vocabulary size: {vocab_size}")

In [16]:
dataframe = [df_3,df_4,df_5,df_6,df_7,df_8]

In [21]:
# Open the file in append mode
with open('_3_4_5_6_7_8_corpus.txt', 'a', encoding='utf-8') as file:
    # Iterate over each DataFrame
    for df in dataframe:
        # Concatenate text data from the specified column
        corpus = ''.join(df['newsText'])
        # Append the corpus to the file
        file.write(corpus + '\n')  # add a newline character for separation between DataFrames

In [22]:
len(corpus)

5259763

In [27]:
with open('_3_4_5_6_7_8_corpus.txt', 'r', encoding='utf-8') as file:
    loaded_corpus_3_8 = file.read()

In [29]:
processed_text_3_8 = preprocess_text(loaded_corpus_3_8)

In [34]:
import pickle

In [35]:
with open('processed_text_3_8.pkl','wb') as file:
    pickle.dump(processed_text_3_8,file)
print("Tokenized list saved to 'processed_text_3_8.pkl'")


Tokenized list saved to 'processed_text_3_8.pkl'


In [45]:
# Load the list from the file using pickle
with open('processed_text_3_8.pkl', 'rb') as file:
    loaded_processed_list = pickle.load(file)

# Print the loaded list
print(loaded_processed_list[500:501])

[['रासस', 'संसद्मा', 'झण्डै', 'दुईतिहाइ', 'बहुमतको', 'शक्तिशाली', 'सरकारले', 'समृद्ध', 'नेपाल', 'सुखी', 'नेपालीको', 'उद्देश्य', 'प्राप्तिका', 'लागि', 'दीर्घकालीन', 'योजनासहित', 'पाँच', 'वर्षे', 'लक्ष्य', 'निर्धारण', 'गरी', 'महत्वपूर्ण', 'लिन्कहरु', 'संसद्मा', 'झण्डै', 'दुईतिहाइ', 'बहुमतको', 'शक्तिशाली', 'सरकारले', 'समृद्ध', 'नेपाल', 'सुखी', 'नेपालीको', 'उद्देश्य', 'प्राप्तिका', 'लागि', 'दीर्घकालीन', 'योजनासहित', 'पाँच', 'वर्षे', 'लक्ष्य', 'निर्धारण', 'गरी', 'महत्वपूर्ण', 'लिन्कहरु', 'पुनःरोजगारीका', 'लागि', 'भाषा', 'परीक्षा', 'भदौमा', 'सरकार', 'अलमलिएको', 'छैन', 'संसद्मा', 'झण्डै', 'दुईतिहाइ', 'बहुमतको', 'शक्तिशाली', 'सरकारले', 'समृद्ध', 'नेपाल', 'सुखी', 'नेपालीको', 'उद्देश्य', 'प्राप्तिका', 'लागि', 'दीर्घकालीन', 'योजनासहित', 'पाँच', 'वर्षे', 'लक्ष्य', 'निर्धारण', 'गरी', 'काम', 'थालनी', 'गरेको', 'छ', 'महिना', 'पनि', 'पुगेको', 'छैन', 'तर', 'सरकारका', 'काम', 'विश्लेषण', 'गर्नुभन्दा', 'विरोध', 'र', 'आलोचनामा', 'रमाउने', 'अधैर्य', 'प्रवृत्ति', 'देखिँदैछ']]


In [13]:

# Print the loaded list
print(loaded_processed_list[1681177:1681179])

[['त्यस्तै', 'विद्यार्थी', 'संगठनका', 'अध्यक्षहरुले', 'अडियो', 'भिडियोमार्फत', 'गठबन्धनका', 'उम्मेदवारहरु', 'विजय', 'गराउन', 'मतदातालाई', 'सार्वजनिक', 'अपिल', 'गर्ने', 'र', 'चुनावी', 'प्रतिस्पर्धामा', 'रहेका', 'दलका', 'शीर्ष', 'नेतृत्व', 'तथा', 'संयुक्त', 'विद्यार्थी', 'संगठनका', 'पूर्वविद्यार्थी', 'अध्यक्षका', 'चुनावी', 'क्षेत्रमा', 'संगठनका', 'अध्यक्षहरु', 'सहभागी', 'हुने', 'पनि', 'निर्णय', 'गरिएको', 'छ'], ['प्रकाशित', 'आश्विन', '२८', '२०७९', '१८३३']]


In [46]:
len(loaded_processed_list)

1681179

In [ ]:
a =loaded_processed_list[:250000]
print(a)

In [47]:
model = Word2Vec.load("word2vec_model_2.model")


In [48]:
model.build_vocab(loaded_processed_list, update=True)


In [49]:
model.train(loaded_processed_list, total_examples=len(loaded_processed_list), epochs=model.epochs)

(241985380, 264301620)

In [52]:
# Save the updated model
model.save("final_word2vec_model_3.model")

In [53]:
model3 = Word2Vec.load("final_word2vec_model_3.model")

In [64]:
import numpy as np

# Load the word vectors
word_vectors = np.load("D:/Major Project/implementation_pr/clean_code/final_word2vec_model_3.model.wv.vectors.npy")

# Check the shape of the word vectors
print(word_vectors.shape)


(158107, 200)


In [65]:
# CLUSTERING THE WORDS IN ORDER TO SEE HOW MODEL CLUSTERS THE WORD WHETHER THEY ARE CONTEXTUAL OR NOT

In [66]:
word_vectors = model3.wv.vectors
words = model3.wv.index_to_key


In [90]:
num_clusters = 200
from sklearn.cluster import KMeans
import numpy as np

In [91]:
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
idx = kmeans.fit_predict(word_vectors)


KeyboardInterrupt: 

In [80]:
word_clusters = {i: [] for i in range(num_clusters)}
for i, word in enumerate(words):
    word_clusters[idx[i]].append(word)


In [85]:
for cluster, words in word_clusters.items():
    with open('clutering.txt','a',encoding='utf-8') as f:
        f.write(f"\nCluster {cluster}:{words}")
 


numpy.ndarray

In [54]:
vocab_size = len(model3.wv.key_to_index)
print(f"New vocabulary size: {vocab_size}")


New vocabulary size: 158107


In [44]:
model1 = Word2Vec.load("nepali_word2vec_1.model")

In [45]:
vocab_size = len(model1.wv.key_to_index)
print(f"New vocabulary size: {vocab_size}")


New vocabulary size: 36741


#### 

In [59]:
model1 = Word2Vec.load("word2vec_model_2.model")
from sklearn.metrics.pairwise import cosine_similarity

# word1 = model.wv['फुल']
# word2 = model.wv['फूल']
word1 = model.wv['प्रजातन्त्र']
word2 = model.wv['राजतन्त्र']

similarity = cosine_similarity([word1], [word2])
print(f"Cosine similarity between 'सुन्दर' and 'फोन': {similarity[0][0]:.4f}")


Cosine similarity between 'सुन्दर' and 'फोन': 0.5049


In [55]:

from sklearn.metrics.pairwise import cosine_similarity

word1 = model3.wv['प्रजातन्त्र']
word2 = model3.wv['राजतन्त्र']
similarity = cosine_similarity([word1], [word2])
print(f"Cosine similarity between 'सुन्दर' and 'फोन': {similarity[0][0]:.4f}")


Cosine similarity between 'सुन्दर' and 'फोन': 0.5187


In [56]:
# Word analogy task
result = model3.wv.most_similar(positive=['राजा', 'महिला'], negative=['पुरुष'])
print("Result of analogy task 'राजा - पुरुष + महिला':")
for word, similarity in result:
    print(f"{word}: {similarity:.4f}")


Result of analogy task 'राजा - पुरुष + महिला':
महेन्द्रले: 0.5822
राजाको: 0.5805
वीरेन्द्रसँग: 0.5679
महेन्द्रबाट: 0.5613
ज्ञानेन्द्रलाई: 0.5449
वरद: 0.5420
पद्मशमशेरले: 0.5355
वंशनाश: 0.5336
राजशाहीको: 0.5254
राज्यकालमा: 0.5228


In [60]:
# Word analogy task
result = model1.wv.most_similar(positive=['राजा', 'महिला'], negative=['पुरुष'])
print("Result of analogy task 'राजा - पुरुष + महिला':")
for word, similarity in result:
    print(f"{word}: {similarity:.4f}")

Result of analogy task 'राजा - पुरुष + महिला':
महेन्द्रले: 0.5351
वीरेन्द्रको: 0.4909
महेन्द्रबाट: 0.4852
वीरविक्रम: 0.4694
बाइसेचौबिसे: 0.4673
वीरेन्द्रले: 0.4648
महेन्द्रको: 0.4642
त्रिभुवनको: 0.4614
त्रिभुवनले: 0.4509
ज्ञानेन्द्रको: 0.4460


In [62]:
# Find similar words to a given word
word = 'राज्य'
similar_words = model.wv.most_similar(word)
print(f"Words similar to '{word}':")
for similar_word, similarity in similar_words:
    print(f"{similar_word}: {similarity:.4f}")


Words similar to 'राज्य':
राजतन्त्रात्मक: 0.5506
एकात्मक: 0.5503
सङ्घात्मक: 0.5415
लक्ष्मणरेखाको: 0.5351
केन्द्रीकृत: 0.5248
लक्ष्मणरेखा: 0.5173
शासन: 0.5089
अङ्गहरू: 0.5084
हुनपुगे: 0.5070
राष्ट्रसङ्घमा: 0.5067


In [57]:
# Find similar words to a given word
word = 'राज्य'
similar_words = model3.wv.most_similar(word)
print(f"Words similar to '{word}':")
for similar_word, similarity in similar_words:
    print(f"{similar_word}: {similarity:.4f}")

Words similar to 'राज्य':
राज्यको: 0.6441
मोर्चाबन्द: 0.6159
लोकतन्त्रात्मक: 0.6045
राज्यसँगै: 0.5753
राज्यभन्दा: 0.5730
संघात्मक: 0.5712
समरूप: 0.5681
व्यवस्थाद्वारा: 0.5665
राज्यका: 0.5662
राज्यराज्यबीच: 0.5652


In [58]:
def normalize_text(text):
    # Remove punctuation and non-Devanagari characters
    text = re.sub(r'[^\u0900-\u097F\s।]', '', text)
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def preprocess_text(text):
    # Normalize text
    normalized_text = normalize_text(text)
    return normalized_text

# input_sentence = "नेपाल एक सुनदेर देश हो।"
input_sentence = "जात्रा', 'आज', 'सकिँदै', 'वरगाछीरोडशेष', 'सडक', 'कालोपत्र', 'हुने', 'विराटनगर', 'मंसिर', '२', 'गते'] "
# input_sentence = " राजनीति सम्भावनासँगै कल्पनाबाहिरको प्रपञ्चको पुलिन्दासमेत बन्न पुगेको छ ।  नेपालको पछिल्लो राजनीतिक परिघटनाले यो अवस्थाको स्पष्ट चित्रण गरेको छ ।  सन्दर्भ सत्ता हस्तान्तरणको हो । "
processed_sentence = preprocess_text(input_sentence)

processed_sentence

'जात्रा आज सकिँदै वरगाछीरोडशेष सडक कालोपत्र हुने विराटनगर मंसिर २ गते'

In [59]:
words = processed_sentence.split()
words

['जात्रा',
 'आज',
 'सकिँदै',
 'वरगाछीरोडशेष',
 'सडक',
 'कालोपत्र',
 'हुने',
 'विराटनगर',
 'मंसिर',
 '२',
 'गते']

In [60]:
# Assuming your Word2Vec model is already loaded
# model = Word2Vec.load("nepali_word2vec_1.model")

word_embeddings = {word: model3.wv[word] for word in words if word in model3.wv}

In [61]:
word_embeddings

{'जात्रा': array([-0.13869497, -0.10688373, -0.85035783, -0.1288541 , -0.01161023,
         0.08458392, -0.5556041 ,  0.26539174, -0.00252168,  0.3767879 ,
        -0.81279945,  0.19643566,  0.04119863, -0.43346933,  0.43166786,
        -0.11549884, -0.1487559 , -0.02139594, -0.2035497 , -0.16317399,
        -0.04638897,  0.08853105,  0.18324442, -0.30969393, -0.28646216,
         0.02764698,  0.11024941, -0.54280585, -0.47910237, -0.43593553,
         0.1764116 ,  0.5087342 , -0.08594426, -0.20815095, -0.5514102 ,
        -0.22758605,  0.85939133, -0.11251764,  0.32722312,  0.351742  ,
        -0.5033325 ,  0.7202566 ,  0.20547605,  0.4313027 ,  0.08329836,
         0.48897216, -0.36399743, -0.35481608, -0.19676094,  0.36847678,
         0.15640435,  0.19448292, -0.7268504 ,  0.22388189, -0.34511936,
        -0.16449144,  0.17540199, -0.04005071,  0.09436304,  0.30903855,
        -0.47789103,  0.48562628,  0.23848228,  0.4981633 , -0.4183004 ,
         0.30878913,  0.40531966,  0.4813

In [62]:
import numpy as np

def get_context_vector(word, words, word_embeddings, window_size=5):
    context_words = []
    word_idx = words.index(word)
    for i in range(max(0, word_idx - window_size), min(len(words), word_idx + window_size + 1)):
        if i != word_idx and words[i] in word_embeddings:
            context_words.append(word_embeddings[words[i]])
    if context_words:
        return np.mean(context_words, axis=0)
    else:
        return np.zeros_like(word_embeddings[word])

context_vectors = {word: get_context_vector(word, words, word_embeddings) for word in words if word in word_embeddings}


In [63]:
from sklearn.metrics.pairwise import cosine_similarity

def flag_incorrect_words(words, word_embeddings, context_vectors, threshold=0.5):
    incorrect_words = []
    for word in words:
        if word in word_embeddings and word in context_vectors:
            similarity = cosine_similarity([word_embeddings[word]], [context_vectors[word]])[0][0]
            if similarity < threshold:
                incorrect_words.append(word)
    return incorrect_words

incorrect_words = flag_incorrect_words(words, word_embeddings, context_vectors)
print("Contextually incorrect words:", incorrect_words)


Contextually incorrect words: ['जात्रा', 'आज', 'सकिँदै', 'सडक', 'कालोपत्र', 'हुने', 'विराटनगर', 'गते']
